In [1]:
import pandas as pd
from google.colab import files
import io

In [2]:
uploaded = files.upload()

Saving traindata.csv to traindata.csv


In [4]:
data = io.BytesIO(uploaded["traindata.csv"])
df = pd.read_csv(data, sep = None, engine = 'python')
df

,id,x1,x2,x3,y
0,1,60,64,0,1
1,2,54,60,11,0
2,3,65,62,22,0
3,4,34,60,0,1
4,5,38,69,21,0
...,...,...,...,...,...
292,293,65,67,0,1
293,294,53,65,12,0
294,295,57,64,1,0
295,296,54,59,7,1


In [9]:
X = df.drop('y', axis = 1)
y = df.y
X = (X - X.min()) / (X.max() - X.min())

df = pd.concat([X,y], axis = 1)
df.head()

,id,x1,x2,x3,y
0,0.000000,0.566038,0.545455,0.000000,1
1,0.003390,0.452830,0.181818,0.211538,0
2,0.006780,0.660377,0.363636,0.423077,0
3,0.010169,0.075472,0.181818,0.000000,1
4,0.013559,0.150943,1.000000,0.403846,0


In [20]:
#mengambil quarter pertama untuk testing dan 3 quarter sisanya untuk training
fold_first_quarter = (df.iloc[0:74].reset_index(drop=True),              df.iloc[74:296].reset_index(drop=True))

#mengambil 50 persen data dengan menyilang masing-masing untuk training dan testing
fold_first_half = (pd.concat([df.iloc[0:74],df.iloc[148:222]]).reset_index(drop=True), pd.concat([df.iloc[74:148],df.iloc[222:296]]).reset_index(drop=True))
fold_second_half = (pd.concat([df.iloc[74:148],df.iloc[148:222]]).reset_index(drop=True), pd.concat([df.iloc[0:74],df.iloc[222:296]]).reset_index(drop=True))

#mengambil quarter akhir untuk testing dan quarter awal hingga quarter ke-tiga untuk training
fold_last_quarter = (df.iloc[0:222].reset_index(drop=True),           df.iloc[222:296].reset_index(drop=True))

In [21]:
#menghitung tingkat ketepatan model yang dibangun
def akurasi(y_pred,y_true):
    n = len(y_pred)
    benar = 0
    for i in range(n):
       if y_pred[i] == y_true[i]:
        benar += 1
    return benar /n * 100

In [10]:
def separate_by_class(df):
  splitted_df = {}
  for kelas in df.y.unique():
    splitted_df[kelas] = df[df.y == kelas].reset_index(drop=True)
  return splitted_df

separate_by_class(df)[0].head()

,id,x1,x2,x3,y
0,0.003390,0.452830,0.181818,0.211538,0
1,0.006780,0.660377,0.363636,0.423077,0
2,0.013559,0.150943,1.000000,0.403846,0
3,0.033898,0.566038,0.090909,0.326923,0
4,0.044068,0.509434,0.272727,0.096154,0


In [12]:
def summarize_data(df):
  return df.describe().loc[['mean', 'std','count']]

summarize_data(df)

,id,x1,x2,x3,y
mean,0.501684,0.423925,0.442608,0.079254,0.737374
std,0.291101,0.205251,0.294194,0.140027,0.440804
count,297.000000,297.000000,297.000000,297.000000,297.000000


In [13]:
def summarize_by_class(df):
  separated = separate_by_class(df)
  summaries = {}
  for kelas in separated:
      summaries[kelas] = summarize_data(separated[kelas])
  return summaries

summarize_by_class(df)

{1:                id          x1          x2          x3      y
 mean     0.493956    0.414060    0.446243    0.054970    1.0
 std      0.289810    0.209167    0.293236    0.114355    0.0
 count  219.000000  219.000000  219.000000  219.000000  219.0,
 0:               id         x1         x2         x3     y
 mean    0.523381   0.451621   0.432401   0.147436   0.0
 std     0.295496   0.192424   0.298537   0.178787   0.0
 count  78.000000  78.000000  78.000000  78.000000  78.0}

#Gaussian Equation

In [16]:
import math

def gaussian_equation(x,mean,std):
  exponen = math.exp(-((x-mean)**2/(2*std**2)))
  return ( 1 / (math.sqrt(2*math.pi) * std)) * exponen

gaussian_equation(1,1,1)

0.3989422804014327

In [22]:
def calculation_probabilites_class(x, summaries):
  n = sum([summaries[kelas].loc['count'].y for kelas in summaries])
  probs = {}
  for kelas in summaries:
    probs[kelas] = summaries[kelas].loc['count'].y / n
    for col in summaries[kelas].columns[:-1]:
        mean, std, count = summaries[kelas][col]
        probs[kelas] *= gaussian_equation(x[col],mean,std)
  return probs

summaries = summarize_by_class(df)

calculation_probabilites_class(df.iloc[0],summaries)

{1: 1.389187927735824, 0: 0.2535301647758332}

In [27]:
def predict(train,test):
  preds = []
  summaries = summarize_by_class(train)
  for _,row in test.iterrows():
    probs = calculation_probabilites_class(row, summaries)
    preds.append(max(probs, key = probs.get))
  return preds

In [30]:
accurate = []

for fold in [fold_first_quarter, fold_first_half, fold_second_half, fold_last_quarter]:
    train, test = fold

    y_test = test.y

    # Add more print statements if needed
    y_preds = predict(train, test)

    accurate.append(akurasi(y_preds, y_test))

print(f'rata-rata akurasi = {sum(accurate) / len(accurate):.2f}%')

rata-rata akurasi = 73.03%
